In [24]:
from PIL import Image
import numpy as np
import plotly.express as px
import tensorflow as tf
import keras.api._v2.keras as keras

In [25]:
image = Image.open("dataset/van_img.jpg")

In [36]:
image = np.asarray(image, dtype=float)

In [37]:
image

array([[[ 42., 109., 198.],
        [ 40., 107., 196.],
        [ 36., 106., 194.],
        ...,
        [ 95., 154., 220.],
        [ 93., 152., 218.],
        [ 98., 157., 223.]],

       [[ 39., 106., 195.],
        [ 38., 105., 194.],
        [ 35., 105., 193.],
        ...,
        [ 94., 153., 219.],
        [ 94., 153., 219.],
        [ 99., 158., 224.]],

       [[ 39., 106., 195.],
        [ 38., 105., 194.],
        [ 36., 106., 192.],
        ...,
        [ 95., 154., 220.],
        [ 95., 154., 220.],
        [ 98., 157., 223.]],

       ...,

       [[140., 126.,  99.],
        [138., 124.,  97.],
        [132., 118.,  91.],
        ...,
        [117., 104.,  85.],
        [126., 114.,  88.],
        [124., 113.,  85.]],

       [[141., 126.,  97.],
        [144., 130., 101.],
        [126., 112.,  85.],
        ...,
        [117., 103.,  90.],
        [118., 105.,  88.],
        [119., 106.,  87.]],

       [[140., 125.,  94.],
        [140., 125.,  94.],
        [123., 1

In [28]:
# px.imshow(image) needs to much power

In [39]:
px.imshow(tf.squeeze(keras.layers.AvgPool2D(8)(tf.expand_dims(image, axis=0))))

In [40]:
px.imshow(tf.squeeze(keras.layers.MaxPool2D(8)(tf.expand_dims(image, axis=0))))

In [71]:
reduced_img = tf.squeeze(keras.layers.AvgPool2D(32)(tf.expand_dims(image, axis=0))).numpy()

In [72]:
reduced_img.shape

(85, 114, 3)

In [73]:
color_arr = np.reshape(reduced_img, (reduced_img.shape[0] * reduced_img.shape[1], reduced_img.shape[2]))

In [89]:
color_arr = color_arr.astype(int)
color_arr[:,]

array([[ 37, 105, 191],
       [ 37, 105, 191],
       [ 37, 105, 191],
       ...,
       [120, 113,  93],
       [116, 104,  86],
       [ 99,  89,  73]])

In [90]:
import plotly.graph_objects as go

colors_hex = ['#%02x%02x%02x' % (r, g, b) for r, g, b in color_arr]

# Create 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=color_arr[:,0],
    y=color_arr[:,1],
    z=color_arr[:,2],
    mode='markers',
    marker=dict(
        size=2,
        color=colors_hex,  # Use the color array
        colorscale='Viridis',  # You can choose other color scales
        opacity=0.8
    )
)])

# Set axis labels
fig.update_layout(scene=dict(xaxis_title='r', yaxis_title='g', zaxis_title='b'))

# Show the plot
fig.show()

In [101]:
color_arr[0]

array([ 37, 105, 191])

In [228]:
import random
def closest_point(p, points):
    """
    Find the closest point to a given point among a list of points.

    Parameters:
    - p: Coordinates of the new point as a NumPy array, e.g., np.array([x, y, z])
    - points: List of points, where each point is a NumPy array of coordinates.

    Returns:
    - The index of the closest point in the list.
    """
    p = np.asarray(p)
    points = np.asarray(points)
    distances = [np.linalg.norm(p - point) for point in points]
    closest_index = np.argmin(distances)
    return closest_index

def k_means(img_arr, k):
    # create random k
    k_arr = [
        [random.Random().randrange(0,255), random.Random().randrange(0,255), random.Random().randrange(0,255)]
        for i in range(k)]
    
    cluster_arr = [[*pixel, closest_point(img_arr[i], k_arr)] for i, pixel in enumerate(img_arr)]
    cluster_arr = np.asarray(cluster_arr)
    changed = True
    while changed:
        changed = False
        for i, curr_k in enumerate(k_arr):
            pixel = [[*pixel[:3], closest_point(pixel[:3], k_arr)] for pixel in cluster_arr if pixel[3] == i]
            mean = np.mean(np.asarray(pixel)[:, :3], axis=0, dtype=int)
    
            k_arr[i] = mean
         
            for i, pixel in enumerate(cluster_arr):

                closest = closest_point(img_arr[i, :3], k_arr)
                
                if closest != cluster_arr[i, 3]:
                    changed = True
                cluster_arr[i] = [*pixel[:3], closest]

    return cluster_arr
    
k_means_result = k_means(color_arr, 3)
k_means_result

array([[ 37, 105, 191,   1],
       [ 37, 105, 191,   1],
       [ 37, 105, 191,   1],
       ...,
       [120, 113,  93,   0],
       [116, 104,  86,   0],
       [ 99,  89,  73,   0]])

In [233]:


# Create 3D scatter plot
px.scatter_3d(
    x=k_means_result[:,0],
    y=k_means_result[:,1],
    z=k_means_result[:,2],
    color=k_means_result[:,3],
    
)